In [1]:
import sys
sys.path.append('/Users/rgao/Documents/Code/research/neurodsp/')

import numpy as np
import scipy as sp
import scipy.io as io
import matplotlib.pyplot as plt
%matplotlib inline

from neurodsp import spectral
import neurodsp as ndsp
from SpectralCV import ecog_pipe as ep

In [22]:
# trying to recreate Lauren's error
fs=1000.
datapath = '/Users/rgao/Documents/Data/NeuroTycho/Propofol/20120802PF_Anesthesia+and+Sleep_Chibi_Toru+Yanagawa_mat_ECoG128/Session%d/'
tf3 = ep.getTimeFile(datapath, 3)
s3start = int(ep.getStart(tf3,0)*fs)
s3end = int(ep.getEnd(tf3,1)*fs)
data = ep.getECoGdata(datapath, 3, 1)
s3end
spectral.scv(data[s3start:s3end], fs, outlierpct=1.)[1]

# the problem is that s3start is a negative number in the timing file, thus slicing doesnt make sense. 
# need to enforce positivity at the starting index

/Users/rgao/anaconda3/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 1000 is greater than input length  = 0, using nperseg = 0
  .format(nperseg, input_length))


IndexError: tuple index out of range